# Advent of Code 2024

Let's try out [ocaml](https://ocaml.org/) this year!  I read a book about Standard ML like 20 years ago, but remember very little.  So there will be a learning curve, but it shouldn't be as bad as [the Advent of APL](https://github.com/jeredw/advent2021/blob/main/Advent%20of%20Code%202021.ipynb).

Is this thing on?

In [1]:
2+2;;

- : int = 4


## Reusable utilities

This section is for stuff I get tired of copying and pasting.

### Ok, what is "core"?

On Day 3, while looking for basic I/O utilities again (is there really no "read entire file" library?), I learned there's an entire alternative standard library written by Jane Street.  This feels weird, but whatever.

In [2]:
#use "topfind" ;;

- : unit = ()
Findlib has been successfully loaded. Additional directives:
  #require "package";;      to load a package
  #list;;                   to list the available packages
  #camlp4o;;                to load camlp4 (standard syntax)
  #camlp4r;;                to load camlp4 (revised syntax)
  #predicates "p,q,...";;   to set these predicates
  Topfind.reset();;         to force that packages will be reloaded
  #thread;;                 to enable threads

- : unit = ()


In [3]:
#require "core" ;;

/Users/jered/.opam/4.12.0/lib/base/base_internalhash_types: added to search path
/Users/jered/.opam/4.12.0/lib/base/base_internalhash_types/base_internalhash_types.cma: loaded
/Users/jered/.opam/4.12.0/lib/base/caml: added to search path
/Users/jered/.opam/4.12.0/lib/base/caml/caml.cma: loaded
/Users/jered/.opam/4.12.0/lib/base/shadow_stdlib: added to search path
/Users/jered/.opam/4.12.0/lib/base/shadow_stdlib/shadow_stdlib.cma: loaded
/Users/jered/.opam/4.12.0/lib/sexplib0: added to search path
/Users/jered/.opam/4.12.0/lib/sexplib0/sexplib0.cma: loaded
/Users/jered/.opam/4.12.0/lib/base: added to search path
/Users/jered/.opam/4.12.0/lib/base/base.cma: loaded
/Users/jered/.opam/4.12.0/lib/fieldslib: added to search path
/Users/jered/.opam/4.12.0/lib/fieldslib/fieldslib.cma: loaded
/Users/jered/.opam/4.12.0/lib/ppx_compare/runtime-lib: added to search path
/Users/jered/.opam/4.12.0/lib/ppx_compare/runtime-lib/ppx_compare_lib.cma: loaded
/Users/jered/.opam/4.12.0/lib/ppx_enumerate/run

In [4]:
open Core

### Oh God, what have I done

Apparently this is an overlay library, and just blows away much of the core language.  Now my `List.map` function is backwards!  That's... incredibly antisocial, actually.  I guess what did I expect from a quantitative finance company.

In [5]:
List.map ((+) 0) [1;2;3]

File "[5]", line 1, characters 9-16:
1 | List.map ((+) 0) [1;2;3]
             ^^^^^^^
Warning 5 [ignored-partial-application]: this function application is partial,
maybe some arguments are missing.


error: compile_error

In [6]:
List.map [1;2;3] ((+) 0)

- : int list = [1; 2; 3]


Hang on, let me just rewrite everything now.

In [7]:
open Core.List

Apparently I need this too.  Have I mentioned it's impossible to Google search for punctuation?  Ugh.

### Reading stuff from files

We seem to have to read stuff from files a lot.

In [8]:
(* Core has a way to do this *)
In_channel.read_lines "day1_example.txt"

- : Base.string Base.list =
["3   4"; "4   3"; "2   5"; "1   3"; "3   9"; "3   3"]


Now there are different combinators, too.  How do these work?

In [9]:
[1;2;3] >>| ((+) 1)

- : int list = [2; 3; 4]


In [10]:
(*let read_numbers_from_file file_name =
  let file = open_in file_name in
  let output = ref [] in
  try
    while true do
      let line = input_line file
               |> Str.split (Str.regexp " +")
               |> List.map int_of_string in
      output := line :: !output
    done;
    assert false
  with
    End_of_file -> List.rev !output *)

let read_numbers_from_file file_name =
  let parse line =
    line |> Str.split (Str.regexp " +") >>| int_of_string in
  In_channel.read_lines file_name >>| parse

val read_numbers_from_file : Base.string -> int list list = <fun>


### This keeps coming up

In [11]:
let pair_of_list xs =
  match xs with
  | [x; y] -> (x, y)
  | _ -> failwith "expecting pair"

val pair_of_list : 'a list -> 'a * 'a = <fun>


### Reading grids

In [12]:
let read_grid_from_file file_name =
  let lines = In_channel.read_lines file_name in
  let rows = length lines in
  let cols = String.length (nth_exn lines 0) in
  let grid = Array.make_matrix rows cols '.' in
  for i = 0 to rows-1 do
    for j = 0 to cols-1 do
      grid.(i).(j) <- String.get (nth_exn lines i) j
    done
  done;
  grid

val read_grid_from_file : Base.string -> char Core.Array.t Core.Array.t =
  <fun>


### Cartesian product without the diagonal

In [13]:
let rec pairs xs = 
  match xs with
  | [] -> []
  | [x] -> []
  | x::rest -> (rest>>|(fun y -> (x,y))) @ pairs rest

val pairs : 'a list -> ('a * 'a) Base.List.t = <fun>


In [14]:
pairs [1;2;3;4;5;6]

- : (int * int) Base.List.t =
[(1, 2); (1, 3); (1, 4); (1, 5); (1, 6); (2, 3); (2, 4); (2, 5); (2, 6);
 (3, 4); (3, 5); (3, 6); (4, 5); (4, 6); (5, 6)]


## Day 1

[Link](https://adventofcode.com/2024/day/1)

### Part 1

They want me to read two columns from a file, sort them, and sum the differences.

In [15]:
let read_pairs_from_file file_name =
  read_numbers_from_file file_name
  >>| pair_of_list

val read_pairs_from_file : Base.string -> (int * int) list = <fun>


In [16]:
read_pairs_from_file "day1_example.txt"

- : (int * int) list = [(3, 4); (4, 3); (2, 5); (1, 3); (3, 9); (3, 3)]


Ok, that was a little ugly, but it could be worse.  Let's use some of this great type matching stuff.

In [17]:
let rec unzip pairs =
  match pairs with
  | [] -> ([], [])
  | (x, y) :: rest ->
    let (xs, ys) = unzip rest in
      (x::xs, y::ys)

val unzip : ('a * 'b) list -> 'a list * 'b list = <fun>


In [18]:
unzip([(1,2); (3,4); (5,6)])

- : int list * int list = ([1; 3; 5], [2; 4; 6])


That was fun.  ~But there's a library for this.~  Update: Ok, core seems to actually call this `zip` and `unzip`...

In [19]:
List.unzip [(1,2); (3,4); (5,6)]

- : int list * int list = ([1; 3; 5], [2; 4; 6])


In [20]:
zip [1;3;5] [2;4;6]

- : (int * int) list Core.List.Or_unequal_lengths.t =
Core.List.Or_unequal_lengths.Ok [(1, 2); (3, 4); (5, 6)]


Oh no, I overwrote `compare` when I did `open Core.List` to try to get the list combinators.  What a mess.

In [21]:
compare

- : ('a -> 'a -> int) -> 'a list -> 'a list -> int = <fun>


In [22]:
Stdlib.compare

- : 'a -> 'a -> int = <fun>


Hmm, post `zip`ping, my list type is infected with `Core.List.Or_unequal_lengths.t` and I can't do unsanctioned list stuff with it.  I guess I have to use `zip_exn` to fail hard.

In [23]:
let day1_part1 pairs =
  let (xs, ys) = unzip pairs in
  (zip_exn (sort xs Stdlib.compare) (sort ys Stdlib.compare)
    >>| fun (x, y) -> abs(x - y))
  |> List.fold ~f:(+) ~init:0

val day1_part1 : (int * int) list -> int = <fun>


~That is actually kind of pretty, although the syntax is meh.~  Let's be real, this is an ugly way to write a simple program.

In [24]:
read_pairs_from_file "day1_example.txt" |> day1_part1

- : int = 11


In [25]:
read_pairs_from_file "day1_input.txt" |> day1_part1

- : int = 1666427


### Part 2

Now we've got to count stuff.  There is no need to be especially clever about this.

In [26]:
let day1_part2 pairs =
  let (xs, ys) = unzip pairs in
  let count x = List.count ys ((=) x) in
    xs
    >>| (fun x -> x * (count x))
    |> List.fold ~f:(+) ~init:0

val day1_part2 : (Core_kernel.Int.t * Core_kernel.Int.t) list -> int = <fun>


In [27]:
read_pairs_from_file "day1_input.txt" |> day1_part2

- : int = 24316233


## Day 2

[Link](https://adventofcode.com/2024/day/2)

### Part 1

Today we have to take adjacent differences of things.

In [28]:
let rec differences list =
  match list with
  | [] -> []
  | [x] -> [] 
  | x :: y :: rest -> x - y :: differences (y :: rest)

val differences : int list -> int list = <fun>


In [29]:
differences [7; 6; 4; 2; 1]

- : int list = [1; 2; 2; 1]


I kind of wish this were point free?  Like I wanted to write `increasing || decreasing`.  Oh well.

In [30]:
let safe levels =
  let decreasing = List.for_all ~f:(fun(x) -> x < 0) in
  let increasing = List.for_all ~f:(fun(x) -> x > 0) in
  let in_range = List.for_all ~f:(fun(x) -> abs(x) >= 1 && abs(x) <= 3) in
  let check xs = (increasing xs || decreasing xs) && in_range xs in
    check (differences levels)

let count safety_check data =
  data
  |> List.filter ~f:safety_check
  |> List.length

val safe : int list -> bool = <fun>


val count : ('a -> bool) -> 'a list -> int = <fun>


In [31]:
read_numbers_from_file "day2_example.txt" |> count safe

- : int = 2


In [32]:
read_numbers_from_file "day2_input.txt" |> count safe

- : int = 442


### Part 2

Now we have to try fudging the data, I guess.

In [33]:
let rec ablate xs =
  match xs with
  | [] -> []
  | x :: rest -> rest :: (List.map ~f:(fun(xs) -> x :: xs)) (ablate rest)

val ablate : 'a list -> 'a list list = <fun>


In [34]:
ablate [1;2;3;4]

- : int list list = [[2; 3; 4]; [1; 3; 4]; [1; 2; 4]; [1; 2; 3]]


In [35]:
let safeish levels =
  safe levels ||
  (ablate levels |> List.exists ~f:safe)

val safeish : int list -> bool = <fun>


In [36]:
read_numbers_from_file "day2_example.txt" |> count safeish

- : int = 4


In [37]:
read_numbers_from_file "day2_input.txt" |> count safeish

- : int = 493


## Day 3

### Part 1

Today we are parsing instructions from strings full of junk.  There is a builtin regex library from, like, the 90s.  Since we're already doing this core stuff, I guess we may as well just use re2 and unlock the _sheer power_ of NFAs.  Stand back.

In [38]:
#require "re2" ;;

/Users/jered/.opam/4.12.0/lib/core_kernel/rope: added to search path
/Users/jered/.opam/4.12.0/lib/core_kernel/rope/rope.cma: loaded
/Users/jered/.opam/4.12.0/lib/re2/c: added to search path
/Users/jered/.opam/4.12.0/lib/re2/c/re2_c.cma: loaded
/Users/jered/.opam/4.12.0/lib/re2: added to search path
/Users/jered/.opam/4.12.0/lib/re2/re2.cma: loaded


In [39]:
let find_day3_instructions s =
  let mul = Re2.create_exn {|mul\((\d+,\d+)\)|} in
  Re2.find_all_exn ~sub:(`Index 1) mul s
  >>| fun (xs) -> (String.split ~on:',' xs
                   >>| int_of_string
                   |> pair_of_list)

val find_day3_instructions : string -> (int * int) list = <fun>


In [40]:
find_day3_instructions "xmul(2,4)%&mul[3,7]!@^do_not_mul(5,5)+mul(32,64]then(mul(11,8)mul(8,5))"

- : (int * int) list = [(2, 4); (5, 5); (11, 8); (8, 5)]


In [41]:
let day3_part1 s =
  find_day3_instructions s
  >>| (fun ((a, b)) -> a * b)
   |> fold ~init:0 ~f:(+)

val day3_part1 : string -> int = <fun>


In [42]:
In_channel.read_all "day3_example.txt" |> day3_part1

- : int = 161


In [43]:
In_channel.read_all "day3_input.txt" |> day3_part1

- : int = 170778545


### Part 2

Oh good, now we get to parse more instructions - we can turn multiplying on and off, how convenient.  Let's use a type thing.

In [44]:
type day3_instruction =
  Day3Mul of int * int
  | Day3Do
  | Day3Dont

type day3_instruction = Day3Mul of int * int | Day3Do | Day3Dont


In [45]:
let parse_day3_program s =
  let opcode = Re2.create_exn {|(don't\(\)|do\(\)|mul\(\d+,\d+\))|} in
  let parse_opcode args =
    match args with
    | "mul" :: x :: y :: rest -> Day3Mul (int_of_string x, int_of_string y)
    | "do" :: rest -> Day3Do
    | "don't" :: rest -> Day3Dont
    | _ -> failwith "unknown instruction" in
  Re2.find_all_exn ~sub:(`Index 1) opcode s
  >>| String.split_on_chars ~on:[','; '('; ')']
  >>| parse_opcode

val parse_day3_program : string -> day3_instruction list = <fun>


In [46]:
parse_day3_program "xmul(2,4)&mul[3,7]!^don't()_mul(5,5)+mul(32,64](mul(11,8)undo()?mul(8,5))"

- : day3_instruction list =
[Day3Mul (2, 4); Day3Dont; Day3Mul (5, 5); Day3Mul (11, 8); Day3Do;
 Day3Mul (8, 5)]


Let's shake things up and evaluate this imperatively.  Mwahaha, purists, take that.

In [47]:
let eval_day3_program s =
  let mul_enabled = ref true in
  let eval instruction =
    match instruction with
    | Day3Do -> mul_enabled := true; 0
    | Day3Dont -> mul_enabled := false; 0
    | Day3Mul (x, y) -> if !mul_enabled then x * y else 0 in
  parse_day3_program s
  >>| eval
  |> fold ~init:0 ~f:(+)

val eval_day3_program : string -> int = <fun>


In [48]:
eval_day3_program "xmul(2,4)&mul[3,7]!^don't()_mul(5,5)+mul(32,64](mul(11,8)undo()?mul(8,5))"

- : int = 48


In [49]:
In_channel.read_all "day3_input.txt" |> eval_day3_program

- : int = 82868252


## Day 4

### Part 1

Today we are doing word searches, which I guess means we need arrays.  Let's see how this works.

In [50]:
let read_wordsearch file_name = read_grid_from_file file_name

val read_wordsearch : Base.string -> char Core.Array.t Core.Array.t = <fun>


In [51]:
read_wordsearch "day4_example.txt"

- : char Core.Array.t Core.Array.t =
[|[|'M'; 'M'; 'M'; 'S'; 'X'; 'X'; 'M'; 'A'; 'S'; 'M'|];
  [|'M'; 'S'; 'A'; 'M'; 'X'; 'M'; 'S'; 'M'; 'S'; 'A'|];
  [|'A'; 'M'; 'X'; 'S'; 'X'; 'M'; 'A'; 'A'; 'M'; 'M'|];
  [|'M'; 'S'; 'A'; 'M'; 'A'; 'S'; 'M'; 'S'; 'M'; 'X'|];
  [|'X'; 'M'; 'A'; 'S'; 'A'; 'M'; 'X'; 'A'; 'M'; 'M'|];
  [|'X'; 'X'; 'A'; 'M'; 'M'; 'X'; 'X'; 'A'; 'M'; 'A'|];
  [|'S'; 'M'; 'S'; 'M'; 'S'; 'A'; 'S'; 'X'; 'S'; 'S'|];
  [|'S'; 'A'; 'X'; 'A'; 'M'; 'A'; 'S'; 'A'; 'A'; 'A'|];
  [|'M'; 'A'; 'M'; 'M'; 'M'; 'X'; 'M'; 'M'; 'M'; 'M'|];
  [|'M'; 'X'; 'M'; 'X'; 'A'; 'X'; 'M'; 'A'; 'S'; 'X'|]|]


In [52]:
let count_matches word grid =
  let count_matches_at i j di dj =
    let len = String.length word in
    let found = ref 1 in
    for n = 0 to len-1 do
      try
        let word_char = String.get word n in
        let grid_char = grid.(i + di * n).(j + dj * n) in
          if Char.(<>) word_char grid_char then found := 0
      with
        Invalid_argument _ -> found := 0
    done; !found in
  let matches = ref 0 in
    let rows = Array.length grid in
    let cols = Array.length grid.(0) in
    let dirs = [(-1, -1); (0, -1); (1, -1);
                (-1,  0);          (1,  0);
                (-1,  1); (0,  1); (1,  1)] in
    for i = 0 to rows-1 do
      for j = 0 to cols-1 do
        let matches_here = dirs
          >>| (fun((di, dj)) -> count_matches_at i j di dj)
          |> fold ~init:0 ~f:(+) in
          matches := !matches + matches_here
      done
    done; !matches

val count_matches : string -> Core.Char.t Core.Array.t Core.Array.t -> int =
  <fun>


Why doesn't the inequality operator `<>` work right for characters?  That's weird.  This Jane Street library is banana pancakes.

In [53]:
read_wordsearch "day4_example.txt" |> count_matches "XMAS"

- : int = 18


In [54]:
read_wordsearch "day4_input.txt" |> count_matches "XMAS"

- : int = 2524


### Part 2

Oh good we have to search a different way.  Let's see...

In [55]:
let count_x_mas grid =
  let count_crossing_matches_at i j =
    try
      let ul = grid.(i-1).(j-1) in
      let ur = grid.(i-1).(j+1) in
      let dl = grid.(i+1).(j-1) in
      let dr = grid.(i+1).(j+1) in
      if Char.(grid.(i).(j) = 'A') &&
        (Char.(ul = 'M') && Char.(dr = 'S') ||
         Char.(ul = 'S') && Char.(dr = 'M')) &&
        (Char.(ur = 'M') && Char.(dl = 'S') ||
         Char.(ur = 'S') && Char.(dl = 'M')) then 1
      else 0
    with
      Invalid_argument _ -> 0 in
  let matches = ref 0 in
    let rows = Array.length grid in
    let cols = Array.length grid.(0) in
    for i = 0 to rows-1 do
      for j = 0 to cols-1 do
        let matches_here = count_crossing_matches_at i j in
          matches := !matches + matches_here
      done
    done; !matches

val count_x_mas : Core.Char.t Core.Array.t Core.Array.t -> int = <fun>


In [56]:
read_wordsearch "day4_input.txt" |>  count_x_mas

- : int = 1873


## Day 5

### Part 1

We're given a ~partial~ (seemingly total?) order as a bunch of pairs `x|y` which means x < y, and then a bunch of strings.  We have to check whether the strings are in order.  We'll just build an adjacency matrix in a hash table to store the ordering.

In [57]:
Hashtbl.of_alist_multi (module Int) [(47, 53); (97, 13); (97, 61); (97, 47)]

- : (Core.Int.t, int list) Core.Hashtbl.t = <abstr>


In [58]:
type day5_problem = {order: (Core.Int.t, int list) Core.Hashtbl.t; test_cases: int list list}

type day5_problem = {
  order : (Core.Int.t, int list) Core.Hashtbl.t;
  test_cases : int list list;
}


In [59]:
let read_day5_problem filename =
  let lines = In_channel.read_lines filename in
  let select char = filter ~f:(fun (line) -> String.contains line char) in
  let order =
    let parse line = String.split ~on:'|' line >>| int_of_string |> pair_of_list in
    lines |> select '|' >>| parse in
  let tests =
    let parse line = String.split ~on:',' line >>| int_of_string in
    lines |> select ',' >>| parse in
  {order = Hashtbl.of_alist_multi (module Int) order;
   test_cases = tests}

val read_day5_problem : Base.string -> day5_problem = <fun>


In [60]:
read_day5_problem "day5_example.txt"

- : day5_problem =
{order = <abstr>;
 test_cases =
  [[75; 47; 61; 53; 29]; [97; 61; 53; 29; 13]; [75; 29; 13];
   [75; 97; 47; 61; 53]; [61; 13; 29]; [97; 13; 75; 29; 47]]}


In [61]:
let order_lookup order x y =
  try
    Hashtbl.find_exn order x
    |> List.exists ~f:((=) y)
  with
  | Stdlib.Not_found | Not_found_s _ -> false

val order_lookup :
  ('a, Core_kernel.Int.t list) Core.Hashtbl.t ->
  'a Core.Hashtbl.key -> Core_kernel.Int.t -> bool = <fun>


In [62]:
let day5_compare order x y =
  if order_lookup order x y then -1
  else if order_lookup order y x then +1
  else 0

val day5_compare :
  (Core_kernel.Int.t, Core_kernel.Int.t list) Core.Hashtbl.t ->
  Core_kernel.Int.t Core.Hashtbl.key -> Core_kernel.Int.t -> int = <fun>


In [63]:
let in_order order xs =
  let len = length xs in
  let valid = ref true in
  (* It would be nicer to write this with a cartesian product that doesn't repeat pairs *)
  for i = 0 to len-2 do
    for j = i+1 to len-1 do
      if day5_compare order (nth_exn xs i) (nth_exn xs j) > 0 then
        valid := false
    done
  done; !valid

val in_order :
  (Core_kernel.Int.t, Core_kernel.Int.t list) Core.Hashtbl.t ->
  Core_kernel.Int.t Core.Hashtbl.key list -> bool = <fun>


In [64]:
let day5_part1 filename =
  let problem = read_day5_problem filename in
  let middle xs = nth_exn xs ((length xs) / 2) in
  let correctly_ordered = problem.test_cases
                        |> filter ~f:(in_order problem.order) in
  correctly_ordered >>| middle |> fold ~init:0 ~f:(+)

val day5_part1 : Base.string -> int = <fun>


In [65]:
day5_part1 "day5_example.txt"

- : int = 143


In [66]:
day5_part1 "day5_input.txt"

- : int = 6242


### Part 2

Now we have to sort stuff.

In [67]:
let day5_part2 filename =
  let problem = read_day5_problem filename in
  let middle xs = nth_exn xs ((length xs) / 2) in
  let incorrectly_ordered = problem.test_cases
                          |> filter ~f:(fun(xs) -> not (in_order problem.order xs)) in
  let compare x y = day5_compare problem.order x y in
  let sort xs = List.sort ~compare:compare xs in
  incorrectly_ordered >>| sort >>| middle |> fold ~init:0 ~f:(+)

val day5_part2 : Base.string -> int = <fun>


In [68]:
day5_part2 "day5_example.txt"

- : int = 123


In [69]:
day5_part2 "day5_input.txt"

- : int = 5169


## Day 6

### Part 1

Today we are plotting lines in a grid I guess.  Haven't we already read grids from files?  Maybe I will go factor out a helper function for that.

In [70]:
let read_map file_name = read_grid_from_file file_name

val read_map : Base.string -> char Core.Array.t Core.Array.t = <fun>


In [71]:
read_map "day6_example.txt"

- : char Core.Array.t Core.Array.t =
[|[|'.'; '.'; '.'; '.'; '#'; '.'; '.'; '.'; '.'; '.'|];
  [|'.'; '.'; '.'; '.'; '.'; '.'; '.'; '.'; '.'; '#'|];
  [|'.'; '.'; '.'; '.'; '.'; '.'; '.'; '.'; '.'; '.'|];
  [|'.'; '.'; '#'; '.'; '.'; '.'; '.'; '.'; '.'; '.'|];
  [|'.'; '.'; '.'; '.'; '.'; '.'; '.'; '#'; '.'; '.'|];
  [|'.'; '.'; '.'; '.'; '.'; '.'; '.'; '.'; '.'; '.'|];
  [|'.'; '#'; '.'; '.'; '^'; '.'; '.'; '.'; '.'; '.'|];
  [|'.'; '.'; '.'; '.'; '.'; '.'; '.'; '.'; '#'; '.'|];
  [|'#'; '.'; '.'; '.'; '.'; '.'; '.'; '.'; '.'; '.'|];
  [|'.'; '.'; '.'; '.'; '.'; '.'; '#'; '.'; '.'; '.'|]|]


Now we need to simulate a guard walking around.  Wow, ocaml's array library is really stilted for fp, or I am just not getting it.  I guess I will just keep it simple and use dumb imperative code.

In [72]:
let find_in_grid p grid =
  let pos = ref (-1, -1)
  and rows = Array.length grid
  and cols = Array.length grid.(0) in
  for i = 0 to rows-1 do
    for j = 0 to cols-1 do
      if p grid.(i).(j) then pos := (i, j)
    done
  done; !pos

val find_in_grid : ('a -> bool) -> 'a Core.Array.t Core.Array.t -> int * int =
  <fun>


In [73]:
read_map "day6_example.txt" |> find_in_grid (Char.(=) '^')

- : int * int = (6, 4)


I am going to have a whole terrible grid library by the end of the month...

In [74]:
let simulate_guard grid =
  let start = find_in_grid (Char.(=) '^') grid
  and turn dir = (snd dir, - fst dir)
  (* I guess Core kept the highly useful polymorphic stuff, but put it in "Poly" libraries... *)
  and count_distinct_positions path =
    Hash_set.Poly.of_list (path >>| fst) |> Hash_set.length in
  let rec move pos dir path =
    let forward = (fst pos + fst dir, snd pos + snd dir) in
    try
      match find path (Poly.(=) (forward, dir)) with
      | Some _ -> path
      | None ->
        match grid.(fst forward).(snd forward) with
        | '#' -> move pos (turn dir) ((pos, turn dir) :: path)
        | _ -> move forward dir ((forward, dir) :: path)
    with
      Invalid_argument _ -> path in
  let guard_path = move start (-1, 0) [(start, (-1, 0))] in
  count_distinct_positions guard_path

val simulate_guard : Core.Char.t Core.Array.t Core.Array.t -> int = <fun>


In [75]:
read_map "day6_example.txt" |> simulate_guard

- : int = 41


In [76]:
read_map "day6_input.txt" |> simulate_guard

- : int = 4663


### Part 2

Now we have to place obstacles to make the guard loop.  The grid is only 130 x 130 so we do not need to be especially clever - we'll just try all the possibilities.  But we should probably use a hash set directly to detect loops now.

In [77]:
let count_ways_to_loop_guard grid =
  let start = find_in_grid (Char.(=) '^') grid
  and turn dir = (snd dir, - fst dir)
  and rows = Array.length grid
  and cols = Array.length grid.(0) in
  let make_history pos = Hash_set.Poly.of_list [(pos, (-1, 0))] in
  let rec detect_loop obstacle pos dir history =
    let forward = (fst pos + fst dir, snd pos + snd dir) in
    try
      match Hash_set.mem history (forward, dir) with
      | true -> true
      | false ->
        match grid.(fst forward).(snd forward) with
        | '#' ->
          Hash_set.add history (pos, turn dir);
          detect_loop obstacle pos (turn dir) history
        | _ when Poly.(=) forward obstacle -> 
          Hash_set.add history (pos, turn dir);
          detect_loop obstacle pos (turn dir) history
        | _ ->
          Hash_set.add history (forward, dir);
          detect_loop obstacle forward dir history
    with
      Invalid_argument _ -> false in
  let ways = ref 0 in
  for i = 0 to rows-1 do
    for j = 0 to cols-1 do
      if (Poly.(<>) (i, j) start) && detect_loop (i,j) start (-1, 0) (make_history start) then incr ways
    done
  done; !ways

val count_ways_to_loop_guard : Core.Char.t Core.Array.t Core.Array.t -> int =
  <fun>


In [78]:
read_map "day6_example.txt" |> count_ways_to_loop_guard

- : int = 6


In [79]:
read_map "day6_input.txt" |> count_ways_to_loop_guard

- : int = 1530


## Day 7

### Part 1

Today we're reading ... possible equations I guess?

In [80]:
let read_equations filename =
  let parse line = String.split_on_chars ~on:[' '; ':'] line
                 |> filter ~f:(fun(s) -> not (String.is_empty s))
                 >>| int_of_string in
  In_channel.read_lines filename >>| parse

val read_equations : Base.string -> int list list = <fun>


In [81]:
read_equations "day7_example.txt"

- : int list list =
[[190; 10; 19]; [3267; 81; 40; 27]; [83; 17; 5]; [156; 15; 6];
 [7290; 6; 8; 6; 15]; [161011; 16; 10; 13]; [192; 17; 8; 14];
 [21037; 9; 7; 18; 13]; [292; 11; 6; 16; 20]]


We have to try inserting + and * operators and evaluating them left-to-right to make the equation work.  We'll use a bitmask to select which operator to insert at each position.  We need to use foldi to apply the operators, but I can't tell from the documentation which parameter is which in the function...

In [82]:
foldi [0; 0; 0; 0] ~init:0 ~f:(fun i result term -> result + i)

- : int = 6


In [83]:
let day7_part1 equations =
  let eval terms ops_mask =
    foldi (tl_exn terms)
      ~init:(hd_exn terms)
      ~f:(fun i result term ->
          match (Int.shift_left 1 i) land ops_mask with
          | 0 -> result + term
          | _ -> result * term) in
  let trial equation =
    let total = hd_exn equation in
    let terms = tl_exn equation in
    let num_operations = (length terms) - 1 in
    let result = ref 0 in
    for ops_mask=0 to (Int.shift_left 1 num_operations)-1 do
      if eval terms ops_mask = total then
        result := total
    done; !result in
  equations >>| trial |> fold ~init:0 ~f:(+)

val day7_part1 : Core_kernel.Int.t list list -> int = <fun>


In [84]:
read_equations "day7_example.txt" |> day7_part1

- : int = 3749


In [85]:
Int.shift_left 1 61 (* ok, I guess we probably have enough precision *)

- : Core.Int.t = 2305843009213693952


In [86]:
read_equations "day7_input.txt" |> day7_part1

- : int = 303876485655


### Part 2

Now we have to support a third type of operator that does digit concatenation.  This could make for some absurdly large numbers... the largest result in our data has 15 digits, so we'll just bail on anything larger.

In [87]:
let day7_part2 equations =
  let digit_cat a b =
    let sa = string_of_int a and sb = string_of_int b in
    if (String.length sa) + (String.length sb) > 15 then a + b
    else int_of_string (sa ^ sb) in
  let eval terms ops_mask =
    foldi (tl_exn terms)
      ~init:(hd_exn terms)
      ~f:(fun i result term ->
          match (ops_mask / (Int.pow 3 i)) mod 3 with
          | 2 -> digit_cat result term 
          | 1 -> result + term
          | _ -> result * term) in
  let trial equation =
    let total = hd_exn equation in
    let terms = tl_exn equation in
    let num_operations = (length terms) - 1 in
    let result = ref 0 in
    for ops_mask=0 to (Int.pow 3 num_operations)-1 do
      if eval terms ops_mask = total then
        result := total
    done; !result in
  equations >>| trial |> fold ~init:0 ~f:(+)

val day7_part2 : Core_kernel.Int.t list list -> int = <fun>


In [88]:
read_equations "day7_example.txt" |> day7_part2

- : int = 11387


In [89]:
read_equations "day7_input.txt" |> day7_part2

- : int = 146111650210682


## Day 8

### Part 1

Today we're trying to map out "antinodes", points which are colinear with two others.  The input is given as a grid, but it seems like we really just want some points.

In [90]:
type day8_antennas =
  {size: int;
   positions: (char, (int * int) list) Core.Hashtbl.Poly.t}

type day8_antennas = {
  size : int;
  positions : (char, (int * int) list) Core.Hashtbl.Poly.t;
}


In [91]:
let read_antennas filename =
  let grid = read_grid_from_file filename in
  let size = Array.length grid in
  let chars_and_points = ref [] in
    for i=0 to size-1 do
      for j=0 to size-1 do
        let here = grid.(i).(j) in
        if Char.(here <> '.') then
          chars_and_points := (here, (i, j)) :: !chars_and_points
      done
    done;
    {size=size;
     positions=Hashtbl.Poly.of_alist_multi !chars_and_points}

val read_antennas : Base.string -> day8_antennas = <fun>


In [92]:
let test_ant = read_antennas "day8_example.txt" in
  Hashtbl.data test_ant.positions

- : (int * int) list list =
[[(1, 8); (2, 5); (3, 7); (4, 4)]; [(5, 6); (8, 8); (9, 9)]]


We're going to want to look at pairs of points... this is the second time we've wanted a `cartesian_product` of distinct items, so let's just write that (I made a `pairs` helper up above).

In [93]:
let find_antinodes antennas =
  let size = antennas.size in
  let in_bounds (x1, y1) = x1 >= 0 && y1 >= 0 && x1 < size && y1 < size in
  let delta (x1, y1) (x2, y2) = (x1 - x2, y1 - y2) in
  let antinodes_for_points ((x1, y1), (x2, y2)) =
    let (dx, dy) = delta (x1, y1) (x2, y2) in
    [(x1 + dx, y1 + dy); (x2 - dx, y2 - dy)] in
  Hashtbl.data antennas.positions >>| pairs
  >>| concat_map ~f:antinodes_for_points |> concat
  |> filter ~f:in_bounds
  |> Hash_set.Poly.of_list |> Hash_set.length

val find_antinodes : day8_antennas -> int = <fun>


In [94]:
read_antennas "day8_example.txt" |> find_antinodes

- : int = 14


In [95]:
read_antennas "day8_input.txt" |> find_antinodes

- : int = 252


### Part 2

Now we have to find even more colinear points.  List comprehensions might be kind of nice... I guess I will use this crummy iota thing instead.

In [96]:
range (-5) 5

- : int list = [-5; -4; -3; -2; -1; 0; 1; 2; 3; 4]


In [97]:
let find_more_antinodes antennas =
  let size = antennas.size in
  let in_bounds (x1, y1) = x1 >= 0 && y1 >= 0 && x1 < size && y1 < size in
  let delta (x1, y1) (x2, y2) = (x1 - x2, y1 - y2) in
  let antinodes_for_points ((x1, y1), (x2, y2)) =
    let (dx, dy) = delta (x1, y1) (x2, y2) in
      (range (-size) size >>| fun d -> (x1 + d * dx, y1 + d * dy))
      |> filter ~f:in_bounds in
  Hashtbl.data antennas.positions >>| pairs
  >>| concat_map ~f:antinodes_for_points |> concat
  |> Hash_set.Poly.of_list |> Hash_set.length

val find_more_antinodes : day8_antennas -> int = <fun>


In [98]:
read_antennas "day8_example.txt" |> find_more_antinodes

- : int = 34


In [99]:
read_antennas "day8_input.txt" |> find_more_antinodes

- : int = 839


## Day 9

### Part 1

We're defragging a hard drive today.  Man, it has been a few years.

In [100]:
type hd_block =
  | Day9File of int
  | Day9FreeSpace

type hd_block = Day9File of int | Day9FreeSpace


In [101]:
let read_hd_map filename =
  let digit c = int_of_char c - int_of_char '0' in
  let unpack index size =
    List.init size ~f:(fun _ ->
      if index mod 2 = 0 then Day9File (index / 2)
      else Day9FreeSpace) in
  In_channel.read_all filename
  |> String.to_list
  >>| digit
  |> concat_mapi ~f:unpack
  |> Array.of_list

val read_hd_map : Base.string -> hd_block Core.Array.t = <fun>


In [102]:
read_hd_map "day9_example.txt"

- : hd_block Core.Array.t =
[|Day9File 0; Day9File 0; Day9FreeSpace; Day9FreeSpace; Day9FreeSpace;
  Day9File 1; Day9File 1; Day9File 1; Day9FreeSpace; Day9FreeSpace;
  Day9FreeSpace; Day9File 2; Day9FreeSpace; Day9FreeSpace; Day9FreeSpace;
  Day9File 3; Day9File 3; Day9File 3; Day9FreeSpace; Day9File 4; Day9File 4;
  Day9FreeSpace; Day9File 5; Day9File 5; Day9File 5; Day9File 5;
  Day9FreeSpace; Day9File 6; Day9File 6; Day9File 6; Day9File 6;
  Day9FreeSpace; Day9File 7; Day9File 7; Day9File 7; Day9FreeSpace;
  Day9File 8; Day9File 8; Day9File 8; Day9File 8; Day9File 9; Day9File 9|]


In [103]:
let defrag hd =
  let len = Array.length hd in
  let i = ref 0 in
  let j = ref (len-1) in
  while !i < !j do
    match hd.(!i), hd.(!j) with
    | Day9File _, _ -> incr i
    | _, Day9FreeSpace -> decr j
    | Day9FreeSpace, Day9File _ -> begin
        hd.(!i) <- hd.(!j);
        hd.(!j) <- Day9FreeSpace;
        incr i; decr j
      end
  done; hd

val defrag : hd_block Core.Array.t -> hd_block Core.Array.t = <fun>


In [104]:
read_hd_map "day9_example.txt" |> defrag

- : hd_block Core.Array.t =
[|Day9File 0; Day9File 0; Day9File 9; Day9File 9; Day9File 8; Day9File 1;
  Day9File 1; Day9File 1; Day9File 8; Day9File 8; Day9File 8; Day9File 2;
  Day9File 7; Day9File 7; Day9File 7; Day9File 3; Day9File 3; Day9File 3;
  Day9File 6; Day9File 4; Day9File 4; Day9File 6; Day9File 5; Day9File 5;
  Day9File 5; Day9File 5; Day9File 6; Day9File 6; Day9FreeSpace;
  Day9FreeSpace; Day9FreeSpace; Day9FreeSpace; Day9FreeSpace; Day9FreeSpace;
  Day9FreeSpace; Day9FreeSpace; Day9FreeSpace; Day9FreeSpace; Day9FreeSpace;
  Day9FreeSpace; Day9FreeSpace; Day9FreeSpace|]


In [105]:
let checksum hd_map =
  Array.foldi hd_map ~init:0 ~f:(fun i total contents ->
    match contents with
    | Day9File num -> total + i * num
    | _ -> total)

val checksum : hd_block Core.Array.t -> int = <fun>


In [106]:
read_hd_map "day9_example.txt" |> defrag |> checksum

- : int = 1928


In [107]:
read_hd_map "day9_input.txt" |> defrag |> checksum

- : int = 6360094256423


### Part 2

Now we need to defrag not quite as badly I guess?  We need to keep track of a few auxiliary data structures for this part, and it is simpler to build them when reading the file, so we'll start over.

In [108]:
type hd = {
  map: hd_block Core.Array.t;
  directory: (int, int * int) Core.Hashtbl.Poly.t;
  free_space_of_size: (int, int list) Core.Hashtbl.Poly.t;
  last_file: int
}

type hd = {
  map : hd_block Core.Array.t;
  directory : (int, int * int) Core.Hashtbl.Poly.t;
  free_space_of_size : (int, int list) Core.Hashtbl.Poly.t;
  last_file : int;
}


In [109]:
let read_hd filename =
  let digit c = int_of_char c - int_of_char '0' in
  let unpack_map index size =
    List.init size ~f:(fun _ ->
      if index mod 2 = 0 then Day9File (index / 2)
      else Day9FreeSpace) in
  let unpack_directory index base size =
    if index mod 2 = 0 then (base + size, Some (index / 2, (base, size)))
    else (base + size, None) in
  let unpack_free_space index base size =
    if index mod 2 = 1 && size > 0 then (base + size, Some (size, base))
    else (base + size, None) in
  let digits = In_channel.read_all filename
             |> String.to_list
             >>| digit in
  let map = digits |> concat_mapi ~f:unpack_map |> Array.of_list in
  let directory = digits |> folding_mapi ~init:0 ~f:unpack_directory |> filter_opt
                |> Hashtbl.Poly.of_alist_exn in
  let free_space = digits |> folding_mapi ~init:0 ~f:unpack_free_space |> filter_opt
                 |> Hashtbl.Poly.of_alist_multi
                 |> Hashtbl.map ~f:(fun xs -> sort ~compare:Int.compare xs) in
  {map=map;
   directory=directory;
   free_space_of_size=free_space;
   last_file=((length digits) - 1) / 2}

val read_hd : Base.string -> hd = <fun>


In [110]:
Hashtbl.data (read_hd "day9_example.txt").free_space_of_size

- : int list list = [[2; 8; 12]; [18; 21; 26; 31; 35]]


This took me way too long because I didn't understand that files could still only move to the left of their original position.  I guess we wouldn't really accomplish defragmenting otherwise...

In [111]:
let defrag2 hd =
  let first_free_of_size before_pos size =
    match Hashtbl.find hd.free_space_of_size size with
    | Some (pos::_) when pos < before_pos -> Some (size, pos)
    | _ -> None in
  let add_free_space size base =
    Hashtbl.add_multi hd.free_space_of_size size base;
    let blocks = Hashtbl.find_exn hd.free_space_of_size size in
    let sorted_blocks = sort blocks ~compare:Int.compare in
    Hashtbl.set hd.free_space_of_size size sorted_blocks in
  let move_file id =
    let (base, size) = Hashtbl.find_exn hd.directory id in
    let valid_sizes = range size 10 in
    let choices = valid_sizes >>| first_free_of_size base |> filter_opt in
    let dest = choices |> min_elt ~compare:(fun (_, x) (_, y) -> Int.compare x y) in
      match dest with
      | Some (dest_size, dest_pos) -> begin
          for i = 0 to size-1 do
            hd.map.(base + i) <- Day9FreeSpace;
            match hd.map.(dest_pos + i) with
            | Day9FreeSpace -> hd.map.(dest_pos + i) <- Day9File id
            | _ -> failwith "overwriting file"
          done;
          Hashtbl.set hd.directory id (dest_pos, size);
          (* the block we just took must be the first *)
          Hashtbl.remove_multi hd.free_space_of_size dest_size;
          (* we don't need this because of the constraint about only moving files to the left *)
          (*add_free_space size base;*)
          if dest_size > size then add_free_space (dest_size - size) (dest_pos + size);
        end
      | None -> () in
  for id=hd.last_file downto 0 do
    move_file id
  done; hd

val defrag2 : hd -> hd = <fun>


In [112]:
read_hd "day9_example.txt" |> defrag2

- : hd =
{map =
  [|Day9File 0; Day9File 0; Day9File 9; Day9File 9; Day9File 2; Day9File 1;
    Day9File 1; Day9File 1; Day9File 7; Day9File 7; Day9File 7;
    Day9FreeSpace; Day9File 4; Day9File 4; Day9FreeSpace; Day9File 3;
    Day9File 3; Day9File 3; Day9FreeSpace; Day9FreeSpace; Day9FreeSpace;
    Day9FreeSpace; Day9File 5; Day9File 5; Day9File 5; Day9File 5;
    Day9FreeSpace; Day9File 6; Day9File 6; Day9File 6; Day9File 6;
    Day9FreeSpace; Day9FreeSpace; Day9FreeSpace; Day9FreeSpace;
    Day9FreeSpace; Day9File 8; Day9File 8; Day9File 8; Day9File 8;
    Day9FreeSpace; Day9FreeSpace|];
 directory = <abstr>; free_space_of_size = <abstr>; last_file = 9}


In [113]:
(read_hd "day9_example.txt" |> defrag2).map |> checksum

- : int = 2858


In [114]:
(read_hd "day9_input.txt" |> defrag2).map |> checksum

- : int = 6379677752410


## Day 10

### Part 1

Hooray, we are searching in a grid.  I love searching in grids.  Yup.

In [115]:
let read_topo_map filename =
  let map = read_grid_from_file filename in
  let digit_of_char c = int_of_char c - int_of_char '0' in
  map |> Array.map ~f:(fun row -> Array.map row digit_of_char)

val read_topo_map : Base.string -> int Core.Array.t Core.Array.t = <fun>


In [116]:
read_topo_map "day10_example.txt"

- : int Core.Array.t Core.Array.t =
[|[|8; 9; 0; 1; 0; 1; 2; 3|]; [|7; 8; 1; 2; 1; 8; 7; 4|];
  [|8; 7; 4; 3; 0; 9; 6; 5|]; [|9; 6; 5; 4; 9; 8; 7; 4|];
  [|4; 5; 6; 7; 8; 9; 0; 3|]; [|3; 2; 0; 1; 9; 0; 1; 2|];
  [|0; 1; 3; 2; 9; 8; 0; 1|]; [|1; 0; 4; 5; 6; 7; 3; 2|]|]


In [117]:
let grid_find_all grid ~x =
  let rows = Array.length grid in
  let cols = Array.length grid.(0) in
  let matches = ref [] in
  for i=0 to rows-1 do
    for j=0 to cols-1 do
      if grid.(i).(j) = x then
        matches := (i, j) :: !matches
      done
    done; !matches

val grid_find_all :
  Core_kernel.Int.t Core.Array.t Core.Array.t ->
  x:Core_kernel.Int.t -> (int * int) list = <fun>


In [118]:
read_topo_map "day10_example.txt" |> grid_find_all ~x:0

- : (int * int) list =
[(7, 1); (6, 6); (6, 0); (5, 5); (5, 2); (4, 6); (2, 4); (0, 4); (0, 2)]


In [119]:
let score_trails map =
  let size = Array.length map in
  let trailheads = grid_find_all map 0 in
  let neighbors (i,j) =
    (if i > 0 && map.(i-1).(j) = 1 + map.(i).(j) then [(i-1, j)] else []) @
    (if j > 0 && map.(i).(j-1) = 1 + map.(i).(j) then [(i, j-1)] else []) @
    (if i < size-1 && map.(i+1).(j) = 1 + map.(i).(j) then [(i+1, j)] else []) @
    (if j < size-1 && map.(i).(j+1) = 1 + map.(i).(j) then [(i, j+1)] else []) in
  let rec search horizon =
    match horizon with
    | [] -> []
    | [(i,j)] ->
        if map.(i).(j) = 9 then [(i, j)]
        else search (neighbors (i, j))
    | (i,j)::rest ->
        if map.(i).(j) = 9 then (i,j) :: search rest
        else search (neighbors (i,j) @ rest) in
  let compare_tuples (a1,b1) (a2,b2) =
    if a1 = a2 then Stdlib.compare b1 b2
    else Stdlib.compare a1 a2 in
  let count_distinct_reachable_9s p =
    search [p] |> dedup_and_sort ~compare:compare_tuples |> length in
  (* part 2 *)
  let count_trails p =
    search [p] |> length in
  let sum = fold ~init:0 ~f:(+) in
  (trailheads >>| count_distinct_reachable_9s |> sum,
   trailheads >>| count_trails |> sum)

val score_trails : Core_kernel.Int.t Core.Array.t Core.Array.t -> int * int =
  <fun>


In [120]:
read_topo_map "day10_example.txt" |> score_trails

- : int * int = (36, 81)


In [121]:
read_topo_map "day10_input.txt" |> score_trails

- : int * int = (593, 1192)


### Part 2

I happened to write my search to find all paths anyway so just added an extra output to the tuple for part 1.

## Day 11

### Part 1

Today is a good day because it is not a grid day.  Instead, today, we've got... stones?  Big stones, apparently.  Is there event a bigint library?  Yes... ok, it's so old that it doesn't work with the current package manager.

In [122]:
#load "nums.cma" ;;

In [123]:
let read_stones filename =
  let trim s =
    let len = String.length s in
    if Char.(s.[len - 1] = '\n') then String.sub s 0 (len - 1)
    else s in
  In_channel.read_all filename
  |> trim
  |> String.split ~on:' '

val read_stones : Base.string -> Base.string list = <fun>


In [124]:
read_stones "day11_example.txt"

- : Base.string list = ["125"; "17"]


In [125]:
let split_digits s =
  let len = String.length s in
  if len mod 2 = 0 then
    Some [Big_int.(string_of_big_int @@ big_int_of_string @@ String.sub s 0 (len/2));
          Big_int.(string_of_big_int @@ big_int_of_string @@ String.sub s (len/2) (len/2))]
  else
    None

val split_digits : string -> string list option = <fun>


In [126]:
let step_stones s =
  if String.(s = "0") then ["1"]
  else match split_digits s with
  | Some split -> split
  | None -> [Big_int.(mult_int_big_int 2024 (big_int_of_string s) |> string_of_big_int)]

val step_stones : Core.String.t -> string list = <fun>


In [127]:
let blink stones = stones |> concat_map ~f:step_stones

val blink : Core.String.t list -> string list = <fun>


In [128]:
let rec iterate n fn acc =
  if n = 0 then acc
  else iterate (n-1) fn (fn acc)

val iterate : Core_kernel.Int.t -> ('a -> 'a) -> 'a -> 'a = <fun>


In [129]:
read_stones "day11_example.txt" |> iterate 6 blink

- : Base.string list =
["2097446912"; "14168"; "4048"; "2"; "0"; "2"; "4"; "40"; "48"; "2024"; "40";
 "48"; "80"; "96"; "2"; "8"; "6"; "7"; "6"; "0"; "3"; "2"]


In [130]:
read_stones "day11_example.txt" |> iterate 25 blink |> length

- : int = 55312


In [131]:
read_stones "day11_input.txt" |> iterate 25 blink |> length

- : int = 220722


### Part 2

Now we have to iterate 75 times.  Let's see if we can just do this.

In [132]:
(* read_stones "day11_input.txt" |> iterate 75 blink |> length *)

Ok, I guess we have to think harder.  How many _distinct_ things do we have after N iterations?

In [133]:
read_stones "day11_input.txt"
|> iterate 25 blink
|> Hash_set.Poly.of_list
|> Hash_set.length

- : int = 380


Great, so let's keep a census of types of stones instead of repeating all that work.  Except Big_ints aren't hashable, because of course... so let's go back and use strings for everything, SIGH.

In [134]:
let update_stone_count ~census ~count kind =
  Hashtbl.update census kind ~f:(fun n -> match n with
    | Some n -> n + count
    | None -> count)

val update_stone_count :
  census:('a, int) Core.Hashtbl.t -> count:int -> 'a Core.Hashtbl.key -> unit =
  <fun>


In [135]:
let make_initial_census stones =
  let census = Hashtbl.create (module String) in
  let _ = stones >>| update_stone_count ~census:census ~count:1 in
  census

val make_initial_census :
  Core.String.t Core.Hashtbl.key list -> (Core.String.t, int) Core.Hashtbl.t =
  <fun>


In [136]:
let blink_smarter census =
  let census' = Hashtbl.create (module String) in
  let _ = Hashtbl.iteri census ~f:(fun ~key ~data ->
    iter (step_stones key) (update_stone_count ~census:census' ~count:data)
  ) in
  census'

val blink_smarter :
  (Core.String.t, int) Core.Hashtbl.t -> (Core.String.t, int) Core.Hashtbl.t =
  <fun>


In [137]:
read_stones "day11_input.txt"
|> make_initial_census
|> iterate 25 blink_smarter
|> Hashtbl.fold ~init:0 ~f:(fun ~key ~data acc -> acc + data)

- : int = 220722


In [138]:
read_stones "day11_input.txt"
|> make_initial_census
|> iterate 75 blink_smarter
|> Hashtbl.fold ~init:0 ~f:(fun ~key ~data acc -> acc + data)

- : int = 261952051690787


## Day 12

### Part 1

Sigh, here we go again with grids.  Today we are flood filling regions I guess.

In [139]:
read_grid_from_file "day12_example.txt"

- : char Core.Array.t Core.Array.t =
[|[|'R'; 'R'; 'R'; 'R'; 'I'; 'I'; 'C'; 'C'; 'F'; 'F'|];
  [|'R'; 'R'; 'R'; 'R'; 'I'; 'I'; 'C'; 'C'; 'C'; 'F'|];
  [|'V'; 'V'; 'R'; 'R'; 'R'; 'C'; 'C'; 'F'; 'F'; 'F'|];
  [|'V'; 'V'; 'R'; 'C'; 'C'; 'C'; 'J'; 'F'; 'F'; 'F'|];
  [|'V'; 'V'; 'V'; 'V'; 'C'; 'J'; 'J'; 'C'; 'F'; 'E'|];
  [|'V'; 'V'; 'I'; 'V'; 'C'; 'C'; 'J'; 'J'; 'E'; 'E'|];
  [|'V'; 'V'; 'I'; 'I'; 'I'; 'C'; 'J'; 'J'; 'E'; 'E'|];
  [|'M'; 'I'; 'I'; 'I'; 'I'; 'I'; 'J'; 'J'; 'E'; 'E'|];
  [|'M'; 'I'; 'I'; 'I'; 'S'; 'I'; 'J'; 'E'; 'E'; 'E'|];
  [|'M'; 'M'; 'M'; 'I'; 'S'; 'S'; 'J'; 'E'; 'E'; 'E'|]|]


Ok, apparently we only have core v0.14.1, because that's the latest version that is in opam?  So I can't copy a matrix.

In [140]:
Array.copy_matrix

error: compile_error

Fine, we'll do it live.

In [141]:
let copy_matrix matrix =
  let rows = Array.length matrix in
  let cols = Array.length matrix.(0) in
  let copy = Array.make_matrix rows cols matrix.(0).(0) in
  for i=0 to rows-1 do
    for j=0 to cols-1 do
      copy.(i).(j) <- matrix.(i).(j)
    done
  done; copy

val copy_matrix :
  'a Core.Array.t Core.Array.t -> 'a Core.Array.t Core.Array.t = <fun>


In [142]:
let find_plant_groups grid =
  (*let plots = Array.copy_matrix grid in*)
  let plots = copy_matrix grid in
  let size = Array.length plots in
  let rec fill plant i j =
    if i < 0 || i > size-1 || j < 0 || j > size-1 then []
    else if Char.(plant = plots.(i).(j)) then begin
      plots.(i).(j) <- '.';
      (i,j) :: (
        fill plant (i-1) j @
        fill plant (i+1) j @
        fill plant i (j-1) @
        fill plant i (j+1))
    end else [] in
  let regions = ref [] in
  for i=0 to size-1 do
    for j=0 to size-1 do
      let plant = plots.(i).(j) in
      if Char.(plant <> '.') then
        regions := (fill plant i j) :: !regions
    done
  done; !regions

val find_plant_groups :
  Core.Char.t Core.Array.t Core.Array.t ->
  (Core_kernel.Int.t * Core_kernel.Int.t) Base.List.t list = <fun>


In [143]:
read_grid_from_file "day12_example.txt" |> find_plant_groups >>| length

- : int list = [3; 5; 14; 13; 1; 11; 13; 10; 14; 4; 12]


In [144]:
let sum = fold ~init:0 ~f:(+)

val sum : int list -> int = <fun>


In [145]:
let fence_cost grid =
  let size = Array.length grid in
  let groups = find_plant_groups grid in
  let fenced_sides (i,j) =
    (if i = 0 || Char.(grid.(i-1).(j) <> grid.(i).(j)) then 1 else 0) +
    (if i = size-1 || Char.(grid.(i+1).(j) <> grid.(i).(j)) then 1 else 0) +
    (if j = 0 || Char.(grid.(i).(j-1) <> grid.(i).(j)) then 1 else 0) +
    (if j = size-1 || Char.(grid.(i).(j+1) <> grid.(i).(j)) then 1 else 0) in
  let perimeter group = group >>| fenced_sides |> sum in
  let area group = length group in
  let cost group = (perimeter group) * (area group) in
  groups >>| cost |> sum

val fence_cost : Core.Char.t Core.Array.t Core.Array.t -> int = <fun>


In [146]:
read_grid_from_file "day12_example.txt" |> fence_cost

- : int = 1930


In [147]:
read_grid_from_file "day12_input.txt" |> fence_cost

- : int = 1304764


### Part 2

Now instead we want to count the edges of each group.  For example, two vertically adjacent plants can share the same west fence, so if a plant needs a west fence but its north neighbor already has a west fence, we'll skip counting it.

In [148]:
let fence_cost2 grid =
  let size = Array.length grid in
  let groups = find_plant_groups grid in
  let needs_fence (i, j) (di, dj) =
    let (i', j') = (i + di, j + dj) in
    if i' < 0 || i' > size - 1 || j' < 0 || j' > size - 1 then true
    else Char.(grid.(i).(j) <> grid.(i').(j')) in
  let cant_share_horizontal_fence (i, j) dir =
    i = 0 ||
    Char.(grid.(i).(j) <> grid.(i - 1).(j)) ||
    not (needs_fence (i - 1, j) dir) in
  let cant_share_vertical_fence (i, j) dir =
    j = 0 ||
    Char.(grid.(i).(j) <> grid.(i).(j - 1)) ||
    not (needs_fence (i, j - 1) dir) in
  let fenced_sides (i, j) =
    (if needs_fence (i, j) (0, -1) &&
        cant_share_horizontal_fence (i, j) (0, -1) then 1
     else 0) +
    (if needs_fence (i, j) (0, 1) &&
        cant_share_horizontal_fence (i, j) (0, 1) then 1
     else 0) +
    (if needs_fence (i, j) (-1, 0) &&
        cant_share_vertical_fence (i, j) (-1, 0) then 1
     else 0) +
    (if needs_fence (i, j) (1, 0) &&
        cant_share_vertical_fence (i, j) (1, 0) then 1
     else 0) in
  let perimeter group = group >>| fenced_sides |> sum in
  let area group = length group in
  let cost group = (perimeter group) * (area group) in
  groups >>| cost |> sum

val fence_cost2 : Core.Char.t Core.Array.t Core.Array.t -> int = <fun>


In [149]:
read_grid_from_file "day12_example1.txt"

- : char Core.Array.t Core.Array.t =
[|[|'A'; 'A'; 'A'; 'A'|]; [|'B'; 'B'; 'C'; 'D'|]; [|'B'; 'B'; 'C'; 'C'|];
  [|'E'; 'E'; 'E'; 'C'|]|]


In [150]:
read_grid_from_file "day12_example1.txt" |> fence_cost2

- : int = 80


In [151]:
read_grid_from_file "day12_example2.txt" |> fence_cost2

- : int = 236


In [152]:
read_grid_from_file "day12_example3.txt" |> fence_cost2

- : int = 368


In [153]:
read_grid_from_file "day12_example.txt" |> fence_cost2

- : int = 1206


In [154]:
read_grid_from_file "day12_input.txt" |> fence_cost2

- : int = 811148
